# Module Import

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive/Colab Notebooks/Final/yolo_tensorflow')

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Input
from keras import backend as K

import cv2
import numpy as np
import os
import pickle
import tensorflow as tf
import yolo.config as cfg

# Configurations

In [0]:
boxes_per_cell = 1
batch_size = 43

classes = cfg.CLASSES
num_class = len(classes)
image_size = cfg.IMAGE_SIZE
cell_size = cfg.CELL_SIZE
output_size = (cell_size * cell_size) * (num_class + boxes_per_cell * 5)
scale = 1.0 * image_size / cell_size
object_scale = cfg.OBJECT_SCALE
noobject_scale = cfg.NOOBJECT_SCALE
class_scale = cfg.CLASS_SCALE
coord_scale = cfg.COORD_SCALE
learning_rate = cfg.LEARNING_RATE

boundary1 = cell_size * cell_size * boxes_per_cell
boundary2 = boundary1 + cell_size * cell_size * boxes_per_cell * 4


# Load Images and Labels

In [0]:
x_trainval = np.load("trainval_images.npy") 
y_trainval = np.load("trainval_label.npy")

# Loss Function Definition

In [0]:
def calc_iou(boxes1, boxes2, scope='iou'):
    """calculate ious
    Args:
      boxes1: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4]  ====> (x_center, y_center, w, h)
      boxes2: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4] ===> (x_center, y_center, w, h)
    Return:
      iou: 4-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    """
    with tf.variable_scope(scope):
        # transform (x_center, y_center, w, h) to (x1, y1, x2, y2)
        
        boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                             boxes1[..., 1] - boxes1[..., 3] / 2.0,
                             boxes1[..., 0] + boxes1[..., 2] / 2.0,
                             boxes1[..., 1] + boxes1[..., 3] / 2.0],
                            axis=-1)

        boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                             boxes2[..., 1] - boxes2[..., 3] / 2.0,
                             boxes2[..., 0] + boxes2[..., 2] / 2.0,
                             boxes2[..., 1] + boxes2[..., 3] / 2.0],
                            axis=-1)

        # calculate the left up point & right down point
        lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
        rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

        # intersection
        intersection = tf.maximum(0.0, rd - lu)
        inter_square = intersection[..., 0] * intersection[..., 1]

        # calculate the boxs1 square and boxs2 square
        square1 = boxes1[..., 2] * boxes1[..., 3]
        square2 = boxes2[..., 2] * boxes2[..., 3]

        union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)

    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)


    
def YOLOloss(labels,predicts):
    
    global boundary1, boundary2, cell_size, boxes_per_cell, num_class, batch_size
    
    
    offset = np.transpose(np.reshape(np.array(
        [np.arange(cell_size)] * cell_size * boxes_per_cell),
        (boxes_per_cell, cell_size, cell_size)), (1, 2, 0))
    
    
    ## Probability
    predict_scales = tf.reshape(predicts[:, :boundary1],[batch_size, cell_size, cell_size, boxes_per_cell])
    response = tf.reshape(labels[..., 0],[batch_size, cell_size, cell_size, 1])
    
    
    ## Coordinate
    predict_boxes = tf.reshape(predicts[:, boundary1:boundary2],[batch_size, cell_size, cell_size, boxes_per_cell, 4])
    boxes = tf.reshape(labels[..., 1:5],[batch_size, cell_size, cell_size, 1, 4])
    boxes = tf.tile(boxes, [1, 1, 1, boxes_per_cell, 1]) 

    
    ## Class
    predict_classes = tf.reshape(predicts[:, boundary2:],[batch_size, cell_size, cell_size, num_class])
    classes = labels[..., 5:]
    
    
    offset = tf.reshape(tf.constant(offset, dtype=tf.float32),[1, cell_size, cell_size, boxes_per_cell])
    offset = tf.tile(offset, [batch_size, 1, 1, 1])
    offset_tran = tf.transpose(offset, (0, 2, 1, 3))
    
    
    predict_boxes_tran = tf.stack(
        [(predict_boxes[..., 0] + offset) / cell_size,
         (predict_boxes[..., 1] + offset_tran) / cell_size,
         tf.square(predict_boxes[..., 2]),
         tf.square(predict_boxes[..., 3])], axis=-1)
    
    iou_predict_truth = calc_iou(predict_boxes_tran, boxes)

    # calculate I tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    object_mask = tf.reduce_max(iou_predict_truth, 3, keep_dims=True)
    object_mask = tf.cast((iou_predict_truth >= object_mask), tf.float32) * response

    # calculate no_I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    noobject_mask = tf.ones_like(object_mask, dtype=tf.float32) - object_mask

    boxes_tran = tf.stack(
        [boxes[..., 0] * cell_size - offset,
         boxes[..., 1] * cell_size - offset_tran,
         tf.sqrt(boxes[..., 2]),
         tf.sqrt(boxes[..., 3])], axis=-1)

    # class_loss
    class_delta = response * (predict_classes - classes)
    class_loss = tf.reduce_mean(
        tf.reduce_sum(tf.square(class_delta), axis=[1, 2, 3]),
        name='class_loss') * class_scale

    # object_loss
    object_delta = object_mask * (predict_scales - iou_predict_truth)
    object_loss = tf.reduce_mean(
        tf.reduce_sum(tf.square(object_delta), axis=[1, 2, 3]),
        name='object_loss') * object_scale

    # noobject_loss
    noobject_delta = noobject_mask * predict_scales
    noobject_loss = tf.reduce_mean(
        tf.reduce_sum(tf.square(noobject_delta), axis=[1, 2, 3]),
        name='noobject_loss') * noobject_scale

    # coord_loss
    coord_mask = tf.expand_dims(object_mask, 4)
    boxes_delta = coord_mask * (predict_boxes - boxes_tran)
    coord_loss = tf.reduce_mean(
        tf.reduce_sum(tf.square(boxes_delta), axis=[1, 2, 3, 4]),
        name='coord_loss') * coord_scale

    
    #print(class_loss,object_loss,noobject_loss,coord_loss)
    
    
    losses = [class_loss,object_loss,noobject_loss,coord_loss]
    total_loss = tf.add_n(losses)
    
    #tf.losses.add_loss(class_loss)
    #tf.losses.add_loss(object_loss)
    #tf.losses.add_loss(noobject_loss)
    #tf.losses.add_loss(coord_loss)

    return total_loss



# Model : InceptionV3 + YOLOloss

In [17]:
# create the base pre-trained model
input_tensor = Input(shape=(448,448,3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(4096, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(output_size)(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 3s 0us/step


In [0]:
model.compile(optimizer='rmsprop', loss=YOLOloss)
model.fit(x=x_trainval, y=y_trainval, batch_size=batch_size, epochs=200, verbose=1, validation_split=0.1)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 2322 samples, validate on 258 samples
Epoch 1/200
2322/2322 [==============================] - 66s 28ms/step - loss: 224.0778 - val_loss: 6.1838
Epoch 2/200
2322/2322 [==============================] - 55s 24ms/step - loss: 6.3967 - val_loss: 8.0997
Epoch 3/200
2322/2322 [==============================] - 55s 24ms/step - loss: 5.2596 - val_loss: 4.5945
Epoch 4/200
2322/2322 [==============================] - 55s 24ms/step - loss: 4.5674 - val_loss: 4.4376
Epoch 5/200
2322/2322 [==============================] - 55s 24ms/step - loss: 4.2210 - val_loss: 4.6848
Epoch 6/200
2322/2322 [==============================] - 55s 24ms/step - loss: 4.0231 - val_loss: 4.9732
Epoch 7/200
2322/2322 [==============================] - 55s 24ms/step - loss: 3.8638 - val_loss: 4.5703
Epoch 8/200
2322/2322 [==============================] - 55s 24ms/step - loss: 3.7760 - val_loss: 4.2373
Epoch 9/200
2322/2322 [===============

# Save and Load Modual

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model_2580train_100epoch.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model_2580train_100epoch.h5")
print("Saved model to disk")

In [6]:
from keras.models import model_from_json

# load json and create model
json_file = open('model_2580train_100epoch.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_2580train_100epoch.h5")
print("Loaded model from disk")

Loaded model from disk


# Test Model on Video 

In [9]:
def image_read(imname, flipped=False):
  global image_size
  image = cv2.imread(imname)
  image = cv2.resize(image, (image_size, image_size))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
  image = (image / 255.0) * 2.0 - 1.0
  if flipped:
      image = image[:, ::-1, :]
  return image

images = []
for index in np.arange(0,226):
  imname = os.path.join(os.getcwd(),'data','cars','frame'+str(index)+'.jpg')
  images.append(image_read(imname))
  if index%10==0:
    print(index)
    
    
images_np = np.stack(images)
np.save('cars.npy',images_np)
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220


In [11]:
image_np = np.load('cars.npy')
y_image = loaded_model.predict(image_np)

print(y_image.shape)
np.save('carPred_2580train_100epoch.npy',y_image)

(226, 343)


In [0]:
# https://blog.csdn.net/qq1483661204/article/details/79681926